In [267]:
import numpy as np
import pandas as pd
#import pandas_profiling as pp
import datetime
import inv_finder

from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)
CSS = """#notebook div.output_subarea {max-width:100%;}""" #changes output_subarea width to 100% (from 100% - 14ex)
HTML('<style>{}</style>'.format(CSS))

import warnings
warnings.filterwarnings('ignore', 'invalid value encountered in greater', RuntimeWarning)
warnings.filterwarnings('ignore', 'invalid value encountered in true_divide', RuntimeWarning)
warnings.filterwarnings('ignore', 'divide by zero encountered in true_divide', RuntimeWarning)


In [243]:
data=pd.read_csv("./data/profiledata.csv",sep=',',na_values=["/////","///","////","//////"])
data=data.replace('C',0)
data['potential_temperature']=data['temperature']+9.8*data['height']/1000

In [244]:
data = data.drop(columns='Unnamed: 0', axis=1)
data['wd'] = pd.to_numeric(data['wd'])
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S')
data = data.sort_values(by=['date', 'height'])


#  利用kahl的方法来获取逆温相关数据。

Beginning at the surface, each temperature profile is scanned upward to locate the first layer in which the temperature increases with altitude. The bottom of this layer identifies the inversion base. The inversion top is defined as the bottom of the first subsequent layer in which the temperature decreases with altitude. Thin negative-lapse layers (<100 m) are ignored if they are embedded within a deeper inversion layer.


Reference: Kahl, J. D. (1990). Characteristics of the low-level temperature inversion along the Alaskan Arctic coast. International Journal of Climatology, 10, 237–548.

数据为探空数据，时间为2013年3月18日-2017年12月31日

In [245]:
invdata=inv_finder.kahl_inversions(data,params)



# 分析探空数据


## 逆温频率, 逆温底高, 逆温厚度, 逆温强度

In [294]:
invdata['inv_strength']=(invdata['dT']/invdata['dZ'])*100 #计算逆温强度 单位：C/100m
t=invdata[(invdata.index.hour==7)|(invdata.index.hour==19)]
t.loc[:,'hour']=t.index.hour
t.loc[(t.index.month==3)|(t.index.month==4)|(t.index.month==5),'season']="spring"
t.loc[(t.index.month==6)|(t.index.month==7)|(t.index.month==8),'season']="summer"
t.loc[(t.index.month==9)|(t.index.month==10)|(t.index.month==11),'season']="autumn"
t.loc[(t.index.month==12)|(t.index.month==1)|(t.index.month==2),'season']="winter"
t=t[t['bh']<500]

### 表1.逆温差统计表（单位：摄氏度）

逆温层的上部温度-下部温度

count字段为发生逆温的天数

In [295]:
t.groupby(['season','hour'])['dT'].describe()

count      mean       std  min  25%  50%    75%   max
season hour                                                       
autumn 7     236.0  1.419915  1.696199  0.1  0.2  0.7  1.900   8.2
       19    207.0  0.679227  0.974183  0.1  0.2  0.3  0.700   5.3
spring 7     280.0  1.723929  1.800945  0.1  0.3  1.1  2.600   9.5
       19    230.0  1.246522  1.561592  0.1  0.2  0.7  1.675   8.0
summer 7     229.0  0.749345  0.723031  0.1  0.2  0.5  1.000   3.5
       19    191.0  0.554450  0.700728  0.1  0.1  0.3  0.700   4.8
winter 7     247.0  2.761134  2.719388  0.1  0.5  1.9  4.750  12.8
       19    205.0  1.263902  1.592835  0.1  0.2  0.6  1.600   9.7

### 表2.逆温起始高度统计表（单位：m）

起始高度限制在500m，剔除掉500m以上的逆温层

In [287]:
t.groupby(['season','hour'])['bh'].describe()

count        mean         std  min  25%    50%    75%    max
season hour                                                              
autumn 7     236.0   64.830508  134.371559  0.0  0.0    0.0    0.0  450.0
       19    207.0   35.024155  108.801388  0.0  0.0    0.0    0.0  450.0
spring 7     280.0  191.785714  165.220686  0.0  0.0  200.0  350.0  450.0
       19    230.0  185.217391  179.758926  0.0  0.0  200.0  350.0  450.0
summer 7     229.0  215.938865  169.161025  0.0  0.0  250.0  350.0  450.0
       19    191.0  113.089005  167.131802  0.0  0.0    0.0  275.0  450.0
winter 7     247.0   33.198381   94.196933  0.0  0.0    0.0    0.0  450.0
       19    205.0   49.024390  127.855711  0.0  0.0    0.0    0.0  450.0

### 表3.逆温厚度统计表（单位：m）

逆温层厚度为逆温层的顶部高度-逆温层的起始高度

In [288]:
t.groupby(['season','hour'])['dZ'].describe()

count        mean         std   min   25%    50%    75%    max
season hour                                                                
autumn 7     236.0   84.322034   56.190842  50.0  50.0   50.0  100.0  300.0
       19    207.0   59.178744   34.314394  50.0  50.0   50.0   50.0  350.0
spring 7     280.0  141.964286  106.582958  50.0  50.0  100.0  200.0  550.0
       19    230.0  117.173913   99.335413  50.0  50.0   50.0  150.0  550.0
summer 7     229.0   96.506550   63.252669  50.0  50.0   50.0  100.0  350.0
       19    191.0   78.795812   53.882086  50.0  50.0   50.0  100.0  300.0
winter 7     247.0  119.433198   93.306921  50.0  50.0  100.0  150.0  550.0
       19    205.0   75.365854   63.132974  50.0  50.0   50.0   50.0  450.0

### 表4.逆温强度统计表（单位：摄氏度/百米）

计算为逆温差与逆温厚度的商

In [289]:
t.groupby(['season','hour'])['inv_strength'].describe()

count      mean       std       min   25%       50%       75%  \
season hour                                                                  
autumn 7     236.0  1.683870  1.876686  0.066667  0.40  0.800000  2.400000   
       19    207.0  1.065236  1.318984  0.066667  0.38  0.600000  1.200000   
spring 7     280.0  1.186156  1.073822  0.066667  0.40  0.900000  1.515714   
       19    230.0  0.972525  0.871125  0.066667  0.40  0.716667  1.300000   
summer 7     229.0  0.729592  0.490233  0.200000  0.40  0.600000  1.000000   
       19    191.0  0.662356  0.602451  0.040000  0.20  0.466667  0.800000   
winter 7     247.0  2.372626  2.104914  0.160000  0.60  1.800000  3.673333   
       19    205.0  1.575827  1.668543  0.200000  0.40  1.000000  2.200000   

              max  
season hour        
autumn 7     10.4  
       19     8.0  
spring 7      7.6  
       19     5.6  
summer 7      2.6  
       19     4.6  
winter 7     10.0  
       19     8.6

### 表5.各季节总数据天数

2013年3月18日-2017年12月31

In [291]:
l=data.groupby('date').count()
l=l[(l.index.hour==7)|(l.index.hour==19)]
l['hour']=l.index.hour
l.loc[(l.index.month==3)|(l.index.month==4)|(l.index.month==5),'season']="spring"
l.loc[(l.index.month==6)|(l.index.month==7)|(l.index.month==8),'season']="summer"
l.loc[(l.index.month==9)|(l.index.month==10)|(l.index.month==11),'season']="autumn"
l.loc[(l.index.month==12)|(l.index.month==1)|(l.index.month==2),'season']="winter"
allszie=l.groupby(['season','hour'])['height'].agg(np.size).to_frame()
#allszie=allszie.to_frame()
allszie.columns=['days']
allszie

days
season hour      
autumn 7      455
       19     453
spring 7      442
       19     443
summer 7      459
       19     460
winter 7      392
       19     392

### 表6.逆温发生频率

发生逆温的天数与观测总天数的商

数据时间段：2013年3月18日-2017年12月31

In [293]:
freq=t.groupby(['season','hour'])['inv_strength'].count()/l.groupby(['season','hour'])['height'].count()
freq=freq.to_frame()
freq.columns=['Frequency']
freq

Frequency
season hour           
autumn 7      0.518681
       19     0.456954
spring 7      0.633484
       19     0.519187
summer 7      0.498911
       19     0.415217
winter 7      0.630102
       19     0.522959